<a href="https://colab.research.google.com/github/tmoura/softexIA/blob/main/DadosDesbalanceados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.compose import make_column_transformer

In [143]:
url = "https://raw.githubusercontent.com/tmoura/machinelearning/master/datasets/bank-full.csv"

dados = pd.read_csv(url)

In [ ]:
dados.head()

In [ ]:
dados.info()

In [ ]:
# dados["default"] = dados["default"].map({"no":0,"yes":1})
# dados["housing"] = dados["housing"].map({"no":0,"yes":1})
# dados["loan"] = dados["loan"].map({"no":0,"yes":1})
# dados["Target"] = dados["Target"].map({"no":0,"yes":1})
# dados.education = dados.education.map({"primary": 0, "secondary":1, "tertiary":2})
# dados.poutcome = dados.poutcome.map({"unknown": 0, "failure":1, "other":2, "success":3})
# dados.contact = dados.contact.map({"cellular": 0, "unknown":1, "telephone":2})
# dados.month = pd.to_datetime(dados.month, format = "%b").dt.month

column_transformer = make_column_transformer((OneHotEncoder(), ['job','marital','education','default','housing','loan','contact','month','poutcome']), remainder='passthrough')

df = column_transformer.fit_transform(dados)

columns_names = column_transformer.get_feature_names_out()

dados = pd.DataFrame(data=df, columns=columns_names)

dados.head()

In [ ]:
dados.remainder__Target.value_counts()

In [ ]:
# Transformação das colunas restantes em numéricas: cada string única é transformada em uma coluna, se o registro existir para aquela coluna ela recebe 1 e zero para as demais colunas.
# dados = pd.get_dummies(dados)

dados.head()

In [ ]:
ax = sns.countplot(x="remainder__Target", data=dados)

In [146]:
X = dados.drop("remainder__Target", axis = 1)

In [147]:
y = dados.remainder__Target

label_encoder = preprocessing.LabelEncoder()
y= label_encoder.fit_transform(y)

In [148]:
# Separando dados de treino e de teste, usando o padrão de 70-30

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=None, stratify=y)

In [ ]:
# Treinando o algoritmo
# Instanciando um algoritmo de classificação
lrmodelo = LogisticRegression(max_iter=3000)
lrmodelo.fit(X_train, y_train)

In [150]:
y_pred = lrmodelo.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print (pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predito'], margins=True))

### Testando o undersampling

In [153]:
nr = NearMiss()

In [154]:
Xnm,ynm = nr.fit_resample(X,y)

In [ ]:
ax = sns.countplot(x=ynm)

In [156]:
X_train, X_test, y_train, y_test = train_test_split(Xnm, ynm, random_state = None, stratify=ynm)

In [157]:
lrmodelo2 = LogisticRegression(max_iter=2000)

In [ ]:
# Treinando o algoritmo

lrmodelo2.fit(X_train, y_train)

In [160]:
y_pred = lrmodelo2.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print (pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predito'], margins=True))

### Testando o SMOTE

In [ ]:
np.bincount(y)

In [164]:
smt = SMOTE()

In [165]:
X, y = smt.fit_resample(X, y)

In [ ]:
ax = sns.countplot(x=y)

In [167]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = None, stratify=y)

In [168]:
lrmodelo3 = LogisticRegression(max_iter=2500)

In [ ]:
lrmodelo3.fit(X_train, y_train)

In [170]:
y_pred = lrmodelo3.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)